<a href="https://colab.research.google.com/github/AgatheMommeja/ClimateCommunity-project/blob/main/PyGMO_Climate_Vulnerability_Notebook_Multi_Objective.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Part 1: Data Preparation

Import libraries.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pygmo as pg
import random
import rasterio
import shapely
import matplotlib.pyplot as plt

In [ ]:
from rasterstats import zonal_stats
from rasterio import features
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
from numpy import float64
from scipy.signal import convolve2d
from pygmo import hypervolume

Set the random seed.

In [ ]:
random.seed(1234)

Set names to filepaths.

In [ ]:
# specify file path to the temperature data
temperature_raster_path = 'input_data/LST_thehague_average_raster.tif'

# specify file path to the rooftop data
buildings_shapefile_path = 'input_data/Platte_daken_classificatie.shp'

# specify file path to the administrative boundary data
admin_boundaries_shapefile_path = 'input_data/DenHaagPC5.shp'

# specify file path to the indicator data
cbs_data_PC5_path = 'input_data/pc5_2020_vol.xlsx'

Read in data.

In [ ]:
# read in administrative boundary data
admin_units = gpd.read_file(admin_boundaries_shapefile_path)

# read in rooftop data
building_rooftops_unfiltered = gpd.read_file(buildings_shapefile_path)

# read in social indicator data
cbs_data_PC5 = pd.read_excel(cbs_data_PC5_path)

Drop unnecesary columns from PC5 shapefile.

In [ ]:
admin_units_cols_to_keep = ['PC5','geometry']

In [ ]:
admin_units = admin_units[admin_units_cols_to_keep]

Merge shapefile with CBS data based on postal code 5.

In [ ]:
admin_units_merged = admin_units.merge(cbs_data_PC5, on='PC5', how='inner')

Modify the data so NaNs are taken out.

In [ ]:
admin_units_merged = admin_units_merged.applymap(lambda x: 0 if x == -99997 else x)

Calculate area for all rooftops.

In [ ]:
# Calculate total rooftop area and green rooftop area for each building
building_rooftops_unfiltered['rooftop_area'] = building_rooftops_unfiltered.area

Define function to filter dataframe.

In [ ]:
def filter_dataframe(df, cols_to_select, flatness_score, threshold):

    # Select rows where column A is higher than the threshold
    mask = df[flatness_score] > threshold
    df_filtered = df.loc[mask]

    # Select only the specified columns from filtered dataframe
    df_selected = df_filtered.loc[:, cols_to_select]

    # return the dataframe that contains only the buildings that are flat enough for green roofs and keep only relevant columns
    return df_selected

Specify input parameters to clean building rooftop dataframe.

In [ ]:
# Specify input parameters for the filter dataframe function
building_rooftop_columns = ['platdak_in','rooftop_area','geometry']
gr_flatness_score_column = 'platdak_in'
gr_threshold = 60

Create filtered dataframe running the filter dataframe function.

In [ ]:
building_rooftops = filter_dataframe(building_rooftops_unfiltered,
                                     building_rooftop_columns,
                                     gr_flatness_score_column,
                                     gr_threshold)

Open the temperature raster and convert to array.

In [ ]:
# convert the temperature raster to an array
with rasterio.open(temperature_raster_path) as src:
    temp_array = src.read()

Define function to create array that represents spatial configurations of green roofs. This is only done to be able to do test runs for the separate objective functions. This has no function within the optimization algorithm.

In [ ]:
def create_array(p):
    return np.random.choice([0, 1], size=building_rooftops.shape[0], p=[1-p, p])

Create spatial configuration of green roofs.

In [ ]:
array_gr = create_array(0.5)

### Part 2: Define Objective Functions

Define function to evaluate temperate reduction.

In [ ]:
def temperature_reduction_function(selected_rooftops):

        # add array to buiding rooftops dataframe to specify which rooftop has a green roof
        building_rooftops_assign = building_rooftops.assign(green_roof=selected_rooftops)

        # select only the rooftops that have been assigned a green roof
        green_rooftops = building_rooftops_assign.loc[building_rooftops_assign['green_roof'] == 1]

        # create tuples of geometry to assign each green roof geometry with a value that is supposed to be burned into the raster.
        geom_value_gr = ((geom,value) for geom, value in zip(green_rooftops.geometry, green_rooftops['rooftop_area']))

        # open temperature raster
        temp_raster = rasterio.open(temperature_raster_path)

        # rasterize vector using the shape and transform data of the origincal temperature raster and burn green areas into raster as raster values
        green_roof_array = features.rasterize(geom_value_gr,
                                    out_shape = temp_raster.shape,
                                    transform = temp_raster.transform,
                                    all_touched = True,
                                    fill = 0,   # background value
                                    merge_alg = MergeAlg.replace,
                                    dtype = float64)

        # convert the temperature raster to an array
        with rasterio.open(temperature_raster_path) as src:
            temp_array = src.read()

        # select only the cells of the raster that are above 20 in the original raster
        non_zero_original_temp = temp_array > 20

        # calculate the average temperature of the origincal raster for all values above 20 degrees celsius
        avg_temp_non_zero_original = np.mean(temp_array[non_zero_original_temp])

        # create a Gaussian kernel that represents the diminishing effect
        def gaussian_kernel(size, sigma):
            x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
            g = np.exp(-((x**2 + y**2)/(2.0 * sigma**2)))
            return g / g.sum()

        kernel_size = 7
        kernel_sigma = 2
        kernel = gaussian_kernel(kernel_size, kernel_sigma)

        # convolve the green_roof_raster with the kernel
        convolved_green_roof_array = convolve2d(green_roof_array, kernel, mode='same', boundary='fill', fillvalue=0)

        # Assume original_raster is your original green roof raster and convolved_raster is the result of the convolution
        restored_green_roof_array = np.where((green_roof_array > 0) & (green_roof_array > convolved_green_roof_array), green_roof_array, convolved_green_roof_array)

        # multiply the convolved raster with the cooling factor
        final_cooling_effect = restored_green_roof_array * 0.0004

        # compute the convolved cooling effect with maximum of 4 degrees celsius
        final_cooling_effect_capped = np.where(final_cooling_effect > 4, 4, final_cooling_effect)

        # compute the new_temperature_raster_kernel by subtracting the cooling_effect_raster from temperature_raster
        new_temperature_raster = temp_array - final_cooling_effect_capped

        # select only the cells of the raster that are above 20 in the new raster
        non_zero_new_temp = new_temperature_raster > 20

        # calculate the average temperature of the new raster for all values above 20 degrees celsius
        avg_temp_non_zero_new = np.mean(new_temperature_raster[non_zero_new_temp])

        # calculate overall temperature reduction across entire raster
        temperature_reduction = avg_temp_non_zero_original - avg_temp_non_zero_new

        def compute_average_above_zero(arr):
                filtered_vals = arr[arr > 0]
                if len(filtered_vals) == 0:
                    return None  # No values greater than 0 found
                else:
                    return np.mean(filtered_vals)

        # calculate the average cooling effect for all the cells that have been cooled to some extent
        cooling_effect_average = compute_average_above_zero(final_cooling_effect_capped)

        return cooling_effect_average

Define function to evaluate coverage.

In [ ]:
def calculate_vulnerability_points(building_rooftops, admin_units_merged, selected_rooftops):
      # add array to buiding rooftops dataframe to specify which rooftop has a green roof
    building_rooftops_assign = building_rooftops.assign(green_roof=selected_rooftops)

    # Calculate green rooftop area
    building_rooftops['green_rooftop_area'] = building_rooftops_assign['rooftop_area'] * building_rooftops_assign['green_roof']

    # Spatially join building rooftops to administrative boundaries
    admin_units_buildings = gpd.sjoin(admin_units_merged, building_rooftops, predicate='intersects')

    # Group by administrative boundary and calculate total rooftop area and green rooftop area
    admin_units_buildings_grouped = admin_units_buildings.groupby('PC5').agg({'rooftop_area': 'sum', 'green_rooftop_area': 'sum', 'geometry': 'first'})

    # Calculate the green roof coverage percentage within each administrative boundary
    admin_units_buildings_grouped['coverage_pct'] = admin_units_buildings_grouped['green_rooftop_area'] / admin_units_buildings_grouped['rooftop_area']

    # Merge admin_boundaries columns to admin_boundaries_with_rooftops_grouped dataframe
    admin_units_df = admin_units_merged[["PC5","INHABITANTS","PEOP_SOC_SEC_BELOW_AOW_AGE"]].merge(admin_units_buildings_grouped, on='PC5')

    # Calculate covered vulnerability points for single indicator
    admin_units_df['cov_vul_points'] = admin_units_df['coverage_pct'] * admin_units_df['INHABITANTS'] * (admin_units_df['PEOP_SOC_SEC_BELOW_AOW_AGE']/admin_units_df['INHABITANTS'])

    # Calculate total vulnerability points for single indicator
    admin_units_df['tot_vul_points'] = admin_units_df['INHABITANTS'] * (admin_units_df['PEOP_SOC_SEC_BELOW_AOW_AGE']/admin_units_df['INHABITANTS'])

    # Calculate level of coverage of vulnerability points
    vulnerability_points_covered = admin_units_df['cov_vul_points'].sum()

    # Calculate level of coverage of vulnerability points
    vulnerability_points_total = admin_units_df['tot_vul_points'].sum()

    vulnerability_coverage = vulnerability_points_covered / vulnerability_points_total

    return vulnerability_coverage

### Part 3: Define Optimization Problem

Create vector to simulate configuration to feed optimization algorithm.

In [ ]:
def encode(x, num_buildings):
    indices = np.argsort(x)[:num_buildings * 2 // 4]
    selected_rooftops = np.zeros(num_buildings, dtype=int)
    selected_rooftops[indices] = 1
    return selected_rooftops

Define optimization problem.

In [ ]:
class GreenRoofProblem:
    def __init__(self, building_rooftops, admin_units_merged, num_rooftops=building_rooftops.shape[0], num_green_roofs=None, heat_stress_reduction_function=None, vulnerable_coverage_function=None):
        self.num_rooftops = num_rooftops
        self.num_green_roofs = num_green_roofs if num_green_roofs else num_rooftops * 2 // 4
        self.building_rooftops = building_rooftops
        self.admin_units_merged = admin_units_merged
        self.heat_stress_reduction_function = heat_stress_reduction_function
        self.calculate_vulnerability_points = calculate_vulnerability_points

    def fitness(self, x):
        selected_rooftops = encode(x, self.num_rooftops)

        heat_stress_reduction = self.heat_stress_reduction_function(selected_rooftops)

        coverage_score = self.calculate_vulnerability_points(self.building_rooftops, self.admin_units_merged, selected_rooftops)

        return [-heat_stress_reduction, -coverage_score]  # Maximize both objectives

    def get_bounds(self):
        return ([0] * self.num_rooftops, [1] * self.num_rooftops)

    def get_nobj(self):
        return 2

    def get_name(self):
        return "Green Roof Placement Problem"

    def get_extra_info(self):
        return f"Number of rooftops: {self.num_rooftops}\nNumber of green roofs: {self.num_green_roofs}"


Create an instance of the GreenRoofProblem class, which is a user-defined problem (UDP) in PyGMO terminology. The resulting UDP object represents the optimization problem, including the decision variables (the configuration of green roofs), the objectives (heat stress reduction and vulnerable population coverage), and the constraints (eg. half of the rooftops must have green roofs). This object can be used to create a PyGMO problem object, which is then used to initialize a population object and solve the problem using an optimization algorithm.

In [ ]:
# Problem definition
udp = GreenRoofProblem(building_rooftops, admin_units_merged,
                       heat_stress_reduction_function=temperature_reduction_function,
                       vulnerable_coverage_function=calculate_vulnerability_points)

In [ ]:
problem = pg.problem(udp)

The following 2 lines of code set up the NSGA-II algorithm with 50 generations and a verbosity level of 2, ready to be used for evolving a population and solving the optimization problem.

- **pg.algorithm(pg.nsga2(gen=50, seed=1234))** creates an instance of the NSGA-II algorithm wrapped in a pg.algorithm object, which is required for using the algorithm in PyGMO. NSGA-II is a popular multi-objective evolutionary algorithm used for solving optimization problems with multiple conflicting objectives.

- **algo.set_verbosity(2)** sets the verbosity level of the algorithm. The verbosity level determines how much information the algorithm provides during the optimization process. A verbosity level of 2 means the algorithm will print a log every 2 generations, giving information about the current generation, the best fitness so far, the average fitness, etc. This can be helpful for tracking the progress of the optimization process and diagnosing any potential issues.

In [ ]:
# set the algorithm and the amount of generations
algo = pg.algorithm(pg.nsga2(gen=2, seed=1234))
algo.set_verbosity(2)

The following line of code creates an instance of a PyGMO population object, which represents a group of candidate solutions for the optimization problem. The population object is initialized with two arguments:

- **Problem**: This is the PyGMO problem object created earlier, which encapsulates the Green Roof Placement Problem's definition, including the objectives, constraints, and bounds. The problem object is used by the population to evaluate the fitness of individuals (candidate solutions).

- **Size**: This argument sets the size of the population, which is the number of individuals (candidate solutions) it contains. In this case, the population size is set to 100, which means the algorithm will work with 100 candidate solutions simultaneously.

In [ ]:
# Population
pop = pg.population(problem, size=8)

The following lines calculate the hypervolume indicator, which is a metric used to assess the quality of a set of solutions in multi-objective optimization. The hypervolume indicator measures the volume of the space dominated by the solutions in the objective space, and it's often used to compare different Pareto fronts in multi-objective optimization.


- **hv = hypervolume(pop)**: This line creates a hypervolume object from the final population pop after the optimization process. The hypervolume object provides methods for computing the hypervolume indicator and related metrics.

- **ref_point = hv.refpoint(offset = 0.1)**: This line computes a reference point for calculating the hypervolume indicator. The reference point is a point in the objective space that is worse than any solution in the population (i.e., it's dominated by all solutions). In multi-objective optimization, solutions are usually evaluated based on how much they dominate the reference point. The offset parameter adds a small margin to the reference point to ensure that it's strictly worse than all solutions.

- **hv_pre_evolve = hv.compute(ref_point)**: This line computes the hypervolume indicator for the population pop with respect to the reference point ref_point. The resulting value represents the volume of the space in the objective space that is dominated by the solutions in pop.

In [ ]:
hv = hypervolume(pop)

In [ ]:
ref_point = hv.refpoint(offset = 0.1)

In [ ]:
hv_pre_evolve = hv.compute(ref_point)

When you run the line pop = algo.evolve(pop), the optimization process using the NSGA-II algorithm starts, evolving the input population pop over multiple generations. After the evolve function call, the evolved population contains candidate solutions that represent better green roof configurations, aiming to maximize heat stress reduction and vulnerable population coverage.

In [ ]:
# Evolve
pop = algo.evolve(pop)

Extract information about the optimization process after it has been completed.

- **uda = algo.extract(pg.nsga2)** is used to extract the underlying algorithm (in this case, NSGA-II) from the PyGMO algorithm object. The extract method returns the specific algorithm instance used inside the pg.algorithm wrapper. The returned UDA (User-Defined Algorithm) object allows you to access the methods and properties of the specific algorithm that are not part of the generic pg.algorithm interface.

- **evolution_fitness_values = uda.get_log()** is used to retrieve the log of the optimization process. The get_log method of the NSGA-II algorithm returns a list of tuples, each containing information about a particular generation during the optimization process. The contents of each tuple depend on the algorithm and the verbosity level, but for NSGA-II with verbosity level 2, the tuple typically includes the generation number, the current best fitness, the average fitness, etc. This log can be useful for analyzing the evolution of the fitness values over the generations and understanding the progress of the optimization process.

In [ ]:
uda = algo.extract(pg.nsga2)
evolution_fitness_values = uda.get_log()

Store the information of the User-Defined Algorithm in a dataframe and write it to a .csv file so it can be accessed later.

In [ ]:
# Create dataframe
evolution_fitness_values_df = pd.DataFrame(evolution_fitness_values, columns=['Verbosity', 'Runs', 'Array'])

Split the column array into the two different optimization objective to store their fitness values.

In [ ]:
evolution_fitness_values_df[['Temperature reduction', 'Vulnerability points covered']] = pd.DataFrame(evolution_fitness_values_df['Array'].to_list())

Drop the column array.

In [ ]:
evolution_fitness_values_df.drop('Array', axis=1, inplace=True)

Store dataframe as .csv file.

In [ ]:
evolution_fitness_values_df.to_csv('evolution_fitness_values.csv', index=False)

Calculate the hypervolume after the evolution of the population. This can provide valuable insight into how the quality of the solution set has improved through the course of the optimization process. By comparing the hypervolume before and after evolution, you can quantify the improvement in the solution set due to the optimization process. The larger the increase in hypervolume, the more the algorithm has been able to improve the solutions.

In [ ]:
hv = pg.hypervolume(pop)
hv_after_evolve = hv.compute(ref_point)

In [ ]:
hv_data = {'Before': [hv_pre_evolve], 'After': [hv_after_evolve]}
hypervolume_df = pd.DataFrame(hv_data)
hypervolume_df.to_csv('hypervolume_values.csv', index=False)

Create a list that stores the different arrays that represent the spatial configurations of the placements of the green roofs.

In [ ]:
# encode the solutions
encoded_solutions = [encode(pop.get_x()[i], udp.num_rooftops) for i in range(len(pop))]

Extract the fitness values from the populations.

In [ ]:
# Extract the fitness values
fitness_values = pop.get_f()

Store the fitness values together with their corresponding spatial configurations in a dataframe.

In [ ]:
# Create a DataFrame to store the results
results_df = pd.DataFrame({'Temperature Reduction (C)': -fitness_values[:,0], 'Vulnerable Coverage': (-fitness_values[:,1]) , 'Configuration': encoded_solutions})

In [ ]:
# Convert the binary values from float to integer to make it more smaller.
results_df['Configuration'] = results_df['Configuration'].apply(lambda x: [int(i) for i in x])

Store the output as .JSON file.

In [ ]:
# Write DataFrame to CSV file
results_df.to_json('optimization_output.json', orient='index', index=True)

Create a dataframe that separately stores the fitness values to be able to visualize the pareto front.

In [ ]:
pareto_front_df = pd.DataFrame(fitness_values, columns=['Temperature reduction', 'Vulnerability points covered'])

In [ ]:
heat_stress_reduction = [-fit[0] for fit in fitness_values]
vulnerable_coverage = [-fit[1] for fit in fitness_values]

Plot the pareto front.

In [ ]:
# Create a scatter plot
plt.scatter(heat_stress_reduction, vulnerable_coverage, marker='o', edgecolors='k', facecolors='none')
plt.xlabel('Heat stress reduction')
plt.ylabel('Vulnerability points covered')
plt.title('Pareto front')

# Show the plot
plt.close()

In [ ]:
obj_values = pop.get_f()

In [ ]:
obj_values2 = pop.get_x()

In [ ]:
obj_values2

array([[0.32235787, 0.80409172, 0.40510254, ..., 0.74386858, 0.23210128,
        0.50377693],
       [0.13013358, 0.73176839, 0.92483529, ..., 0.68623463, 0.97094234,
        0.47833066],
       [0.27178049, 0.5907202 , 0.06567685, ..., 0.79351146, 0.21561793,
        0.66068373],
       ...,
       [0.13013358, 0.73759282, 0.9275513 , ..., 0.6857043 , 0.99516514,
        0.49354768],
       [0.32235787, 0.27659525, 0.40509691, ..., 0.74386858, 0.23736142,
        0.50377693],
       [0.13013358, 0.73176839, 0.92483529, ..., 0.68566633, 0.97094234,
        0.47800725]])

In [ ]:
front_indices = pg.non_dominated_front_2d(obj_values)

In [ ]:
front_indices

array([2, 1, 0], dtype=uint64)

In [ ]:
# Extract the non-dominated front individuals
non_dominated_front = pop[front_indices]

# Access the decision variables and objective values of the non-dominated front
front_variables = non_dominated_front.get_x()
front_objectives = non_dominated_front.get_f()